<a href="https://colab.research.google.com/github/jumbokh/Computer-Vision/blob/main/notebooks/face_recognition_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## https://github.com/Jim-Chang/KodingWork/tree/master/deep_learning/face_recognition

In [1]:
!pip install opencv-python numpy face-recognition pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=dba6aae785dd7e53d0aa87066c5c1c943c955de1c249bc4bf69378c43eba0c05
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [2]:
import cv2
import numpy as np
import face_recognition
import os

In [3]:
known_face_list = [
    {
        'name': 'Hyun Bin',
        'filename': '玄彬.jpeg',
        'encode': None,
    },
    {
        'name': 'Son Ye Jin',
        'filename': '孫藝珍.jpeg',
        'encode': None,
    },
]

In [4]:
# load image data
for data in known_face_list:
    img = cv2.imread(data['filename'])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    data['encode'] = face_recognition.face_encodings(img)[0]

known_face_encodes = [data['encode'] for data in known_face_list]
tolerance = 0.6

In [5]:
test_fn_list = ['孫藝珍-t1.jpeg', '孫藝珍-t2.jpeg', '孫藝珍-t3.jpeg', '玄彬+孫藝珍.jpeg']

## 如何儲存 Face Encoding 加速載入？

In [6]:
import pickle

In [7]:
# save known_face_list to dat file
with open('faces.dat', 'wb') as f:
    pickle.dump(known_face_list, f)

In [8]:
# load known_face_list from dat file
with open('faces.dat', 'rb') as f:
    known_face_list = pickle.load(f)

## 發現有時會誤判人臉？改用 CNN 模型

In [9]:
import time

for fn in test_fn_list:
    img = cv2.imread(fn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    _t = time.time()

    face_recognition.face_locations(img) # use HOG model to detect face locations

    _t1 = time.time()

    face_recognition.face_locations(img, model='cnn') # use CNN model to detect face locations

    print(f'HOG: {round(_t1 - _t, 2)} secs, CNN: {round(time.time() - _t1, 2)} secs')


HOG: 0.35 secs, CNN: 1.26 secs
HOG: 0.3 secs, CNN: 0.05 secs
HOG: 0.23 secs, CNN: 0.21 secs
HOG: 0.51 secs, CNN: 0.41 secs


## 準確度不夠？增加特徵點數量

### 使用 68 個特徵點

In [10]:
known_face_list = [
    {
        'name': 'Lee',
        'filename': 'lee.jpg',
        'encode': None,
    },
    {
        'name': 'Pan',
        'filename': 'pan.jpg',
        'encode': None,
    },
]

test_fn_list = ['lee-t1.jpg', 'lee-t2.jpg', 'pan-t1.jpg', 'pan-t2.jpg']

# load image data by large model of face landmarks
for data in known_face_list:
    img = cv2.imread(data['filename'])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    data['encode'] = face_recognition.face_encodings(img, model='large')[0]  # use large model of face landmarks

known_face_encodes = [data['encode'] for data in known_face_list]

# face recognition
for fn in test_fn_list:
    img = cv2.imread(fn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    cur_face_locs = face_recognition.face_locations(img)
    cur_face_encodes = face_recognition.face_encodings(img, cur_face_locs, model='large')  # use large model of face landmarks

    for cur_face_encode in cur_face_encodes:
        face_distance_list = face_recognition.face_distance(known_face_encodes, cur_face_encode)

        min_distance_index = np.argmin(face_distance_list)
        if face_distance_list[min_distance_index] < tolerance:
            result = known_face_list[min_distance_index]['name']
        else:
            result = 'unknown'

        distance_with_name_list = [(face_data['name'], round(distance, 4)) for face_data, distance in zip(known_face_list, face_distance_list)]
        print(f'辨識檔案: {fn}, 辨識結果: {result}, 特徵距離: {distance_with_name_list}, 相差: {round(abs(distance_with_name_list[0][1] - distance_with_name_list[1][1]), 4)}')

辨識檔案: lee-t1.jpg, 辨識結果: Lee, 特徵距離: [('Lee', 0.3777), ('Pan', 0.4614)], 相差: 0.0837
辨識檔案: lee-t2.jpg, 辨識結果: Lee, 特徵距離: [('Lee', 0.3033), ('Pan', 0.5545)], 相差: 0.2512
辨識檔案: pan-t1.jpg, 辨識結果: Pan, 特徵距離: [('Lee', 0.4679), ('Pan', 0.2946)], 相差: 0.1733
辨識檔案: pan-t2.jpg, 辨識結果: Lee, 特徵距離: [('Lee', 0.3897), ('Pan', 0.4047)], 相差: 0.015


### 使用 5 個特徵點

In [11]:
known_face_list = [
    {
        'name': 'Lee',
        'filename': 'lee.jpg',
        'encode': None,
    },
    {
        'name': 'Pan',
        'filename': 'pan.jpg',
        'encode': None,
    },
]

test_fn_list = ['lee-t1.jpg', 'lee-t2.jpg', 'pan-t1.jpg', 'pan-t2.jpg']

# load image data by large model of face landmarks
for data in known_face_list:
    img = cv2.imread(data['filename'])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    data['encode'] = face_recognition.face_encodings(img, model='small')[0]  # use small model of face landmarks

known_face_encodes = [data['encode'] for data in known_face_list]

# face recognition
for fn in test_fn_list:
    img = cv2.imread(fn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    cur_face_locs = face_recognition.face_locations(img)
    cur_face_encodes = face_recognition.face_encodings(img, cur_face_locs, model='small')  # use small model of face landmarks

    for cur_face_encode in cur_face_encodes:
        face_distance_list = face_recognition.face_distance(known_face_encodes, cur_face_encode)

        min_distance_index = np.argmin(face_distance_list)
        if face_distance_list[min_distance_index] < tolerance:
            result = known_face_list[min_distance_index]['name']
        else:
            result = 'unknown'

        distance_with_name_list = [(face_data['name'], round(distance, 4)) for face_data, distance in zip(known_face_list, face_distance_list)]
        print(f'辨識檔案: {fn}, 辨識結果: {result}, 特徵距離: {distance_with_name_list}, 相差: {round(abs(distance_with_name_list[0][1] - distance_with_name_list[1][1]), 4)}')

辨識檔案: lee-t1.jpg, 辨識結果: Lee, 特徵距離: [('Lee', 0.4014), ('Pan', 0.4467)], 相差: 0.0453
辨識檔案: lee-t2.jpg, 辨識結果: Lee, 特徵距離: [('Lee', 0.3171), ('Pan', 0.533)], 相差: 0.2159
辨識檔案: pan-t1.jpg, 辨識結果: Pan, 特徵距離: [('Lee', 0.4501), ('Pan', 0.3395)], 相差: 0.1106
辨識檔案: pan-t2.jpg, 辨識結果: Lee, 特徵距離: [('Lee', 0.3822), ('Pan', 0.3957)], 相差: 0.0135
